<i>The Battle of Neighborhoods (Week 2) by Dr K.R.S Dulanjalee</i>

# <center><span style="color:green">Segmenting and Clustering Neighborhoods in Toronto</span></center>

## 1. <span style="color:orange">Import relevant libraries</span>

In [227]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import pylab as pl
import seaborn as sns
import scipy.optimize as opt
%matplotlib notebook
%matplotlib inline


import json 

!pip install geopy  
from geopy.geocoders import Nominatim

import requests 
from pandas.io.json import json_normalize 

from sklearn.cluster import KMeans

!pip install folium 
import folium 

## 2. <span style="color:orange">Load Data and make a Data Frame</span>

In [228]:
# The code was removed by Watson Studio for sharing.

,Index,Postal Codes,Latitudes,Longitudes,Borough
0,1,V0A1K1,50.339434,-115.858121,Fairmont Hot Springs
1,2,V0B0V0,49.218994,-122.949596,Burnaby
2,3,V0C2B0,56.247431,-120.879999,Fort St. John
3,4,V0E2Z7,50.711182,-119.261332,Salmon Arm
4,6,V0H0H0,49.160556,-123.114890,Richmond


In [229]:
BC_df.shape

(278, 5)

In [230]:
BC_df.columns

Index(['Index', 'Postal Codes', 'Latitudes', 'Longitudes', 'Borough'], dtype='object')

In [231]:
BC_df = BC_df.drop(['Index'], axis = 1) 
BC_df.head()

,Postal Codes,Latitudes,Longitudes,Borough
0,V0A1K1,50.339434,-115.858121,Fairmont Hot Springs
1,V0B0V0,49.218994,-122.949596,Burnaby
2,V0C2B0,56.247431,-120.879999,Fort St. John
3,V0E2Z7,50.711182,-119.261332,Salmon Arm
4,V0H0H0,49.160556,-123.114890,Richmond


In [232]:
A = BC_df[BC_df['Borough'] == 'Campbell River']
A

,Postal Codes,Latitudes,Longitudes,Borough
263,V9H0E6,49.960339,-125.236511,Campbell River
264,V9H0E7,49.961222,-125.238884,Campbell River
270,V9W0E9,45.730295,-76.603025,Campbell River
271,V9W3S4,49.906397,-125.195014,Campbell River


In [233]:
BC_df = BC_df.drop([263, 264, 270,271])
BC_df.tail(30)

,Postal Codes,Latitudes,Longitudes,Borough
244,V7X0P4,49.186800,-123.132399,Richmond
245,V7X1J1,49.286137,-123.119745,Vancouver
246,V7X1L7,49.286136,-123.119745,Vancouver
247,V8A0C7,49.886060,-124.544084,Powell River
248,V8B0E3,49.732770,-123.141706,Squamish
249,V8E0Y0,50.120881,-122.936620,Whistler
250,V8J4N7,54.310921,-130.304363,Prince Rupert
251,V8L0H9,48.411411,-123.495567,Colwood
252,V8S0C3,49.699478,-123.154697,Squamish
253,V8T0E1,48.448495,-123.336286,Victoria


In [234]:
B = BC_df[BC_df['Borough'] == 'Campbell River']
B

,Postal Codes,Latitudes,Longitudes,Borough


In [235]:
BC_df.dtypes

Postal Codes     object
Latitudes       float64
Longitudes      float64
Borough          object
dtype: object

## 3. <span style="color:orange">Visualization</span>

In [236]:
address = 'British Columbia'

geolocator = Nominatim(user_agent="British Columbia_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of British Columbia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of British Columbia are 55.001251, -125.002441.


In [237]:
map_BritishColumbia = folium.Map(location=[latitude, longitude], zoom_start=5)

for lat, lng, borough in zip(BC_df['Latitudes'], BC_df['Longitudes'], BC_df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_BritishColumbia)  
    
map_BritishColumbia

## 4. <span style="color:orange">Segment and cluster only the neighborhoods in Surrey</span>

In [238]:
surrey_data = BC_df[BC_df['Borough'] == 'Surrey'].reset_index(drop=True)
surrey_data.head()

,Postal Codes,Latitudes,Longitudes,Borough
0,V2S0Z5,49.186697,-122.809079,Surrey
1,V2S9J2,49.107889,-122.807934,Surrey
2,V2W5Y4,49.144302,-122.845344,Surrey
3,V3A8X9,49.043329,-122.784200,Surrey
4,V3E6Y6,49.156208,-122.852509,Surrey


In [239]:
surrey_data.shape

(41, 4)

In [240]:
map_surrey= folium.Map(location=[latitude, longitude], zoom_start=20)


for lat, lng, label in zip(surrey_data['Latitudes'], surrey_data['Longitudes'], surrey_data['Postal Codes']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_surrey)  
    
map_surrey

## 4. <span style="color:orange">Connecting to Foursquare API</span> 

In [241]:
# The code was removed by Watson Studio for sharing.

In [242]:
surrey_data.loc[10,'Postal Codes']

' V3S0S9'

In [243]:
V3S0S9_latitude = surrey_data.loc[10, 'Latitudes']
V3S0S9_longitude = surrey_data.loc[10,'Longitudes']

V3S0S9_PostalCode = surrey_data.loc[10,'Postal Codes'] 

print('Latitude and longitude values of {} are {}, {}.'.format(V3S0S9_PostalCode, 
                                                              V3S0S9_latitude, 
                                                               V3S0S9_longitude))

Latitude and longitude values of  V3S0S9 are 49.054790999999994, -122.790462.


In [244]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    V3S0S9_latitude, 
    V3S0S9_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=TQ3XJQJQJ13M3N0M1HFN5D4XAYYX20YYDANQXSMIWYZWETOF&client_secret=JJSDCPNQRXD0OYSAFQ10OK0SSZY4LFGWDVNC341IZ4MNOEIM&v=20180605&ll=49.054790999999994,-122.790462&radius=1000&limit=100'

In [245]:
results = requests.get(url).json()

In [246]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [247]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



,name,categories,lat,lng
0,Thrifty Foods,Grocery Store,49.047880,-122.785404
1,Mink Chocolate Cafe,Café,49.047582,-122.785037
2,Everything Wine,Wine Shop,49.047787,-122.785590
3,COBS Bread,Bakery,49.056173,-122.800673
4,The Shops at Morgan Crossing,Shopping Mall,49.047916,-122.784636


In [248]:
nearby_venues.shape

(36, 4)

In [249]:
nearby_venues.columns

Index(['name', 'categories', 'lat', 'lng'], dtype='object')

In [250]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood Postal Code', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [251]:
surrey_venues = getNearbyVenues(names=surrey_data['Postal Codes'],
                                   latitudes=surrey_data['Latitudes'],
                                   longitudes=surrey_data['Longitudes']
                                  )
surrey_venues

 V2S0Z5
 V2S9J2
 V2W5Y4
 V3A8X9
 V3E6Y6
 V3N0N6
 V3R0L3
 V3R9M9
 V3S0H5
 V3S0M6
 V3S0S9
 V3S2J6
 V3S3Y3
 V3S6P4
 V3S8K2
 V3S8T5
 V3S8W2
 V3S9W7
 V3T0K5
 V3T1P8
 V3T2K7
 V3T4B6
 V3W3M6
 V3W4K8
 V3W5G4
 V3W5R2
 V3X1A5
 V3X3W4
 V3Z1N4
 V3Z7B3
 V4A8X6
 V4B4Z7
 V4B4Z8
 V4M4V8
 V4N2E6
 V4N4X6
 V4P0N7
 V4P3J4
 V4P4P1
 V6N6N3
 V6Z0T3


,Neighborhood Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,V2S0Z5,49.186697,-122.809079,Disney store,49.188895,-122.803649,Toy / Game Store
1,V2S0Z5,49.186697,-122.809079,Apple Guildford Town Centre,49.189299,-122.803620,Electronics Store
2,V2S0Z5,49.186697,-122.809079,Hanako,49.186283,-122.802545,Sushi Restaurant
3,V2S0Z5,49.186697,-122.809079,Landmark Cinemas 12 Guildford,49.186949,-122.804454,Multiplex
4,V2S0Z5,49.186697,-122.809079,Akasaka,49.185884,-122.803315,Japanese Restaurant
5,V2S0Z5,49.186697,-122.809079,The Lego Store,49.189312,-122.803782,Toy / Game Store
6,V2S0Z5,49.186697,-122.809079,H&M,49.189090,-122.804715,Clothing Store
7,V2S0Z5,49.186697,-122.809079,Purdys Chocolatier,49.189510,-122.804908,Gift Shop
8,V2S0Z5,49.186697,-122.809079,London Drugs,49.189902,-122.805193,Pharmacy
9,V2S0Z5,49.186697,-122.809079,McDonald's,49.188579,-122.805013,Fast Food Restaurant


In [252]:
surrey_venues.shape


(320, 7)

In [253]:
surrey_venues.head()

,Neighborhood Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,V2S0Z5,49.186697,-122.809079,Disney store,49.188895,-122.803649,Toy / Game Store
1,V2S0Z5,49.186697,-122.809079,Apple Guildford Town Centre,49.189299,-122.803620,Electronics Store
2,V2S0Z5,49.186697,-122.809079,Hanako,49.186283,-122.802545,Sushi Restaurant
3,V2S0Z5,49.186697,-122.809079,Landmark Cinemas 12 Guildford,49.186949,-122.804454,Multiplex
4,V2S0Z5,49.186697,-122.809079,Akasaka,49.185884,-122.803315,Japanese Restaurant


In [254]:
surrey_onehot = pd.get_dummies(surrey_venues[['Venue Category']], prefix="", prefix_sep="")

surrey_onehot['Neighborhood Postal Code'] = surrey_venues['Neighborhood Postal Code'] 

fixed_columns = [surrey_onehot.columns[-1]] + list(surrey_onehot.columns[:-1])
surrey_onehot = surrey_onehot[fixed_columns]

surrey_onehot.head()

,Neighborhood Postal Code,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Big Box Store,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Stop,Business Service,Café,Carpet Store,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Food & Drink Shop,Food Court,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Library,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Multiplex,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoothie Shop,Spa,Sporting Goods Shop,Stables,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Thai Restaurant,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Wine Shop
0,V2S0Z5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,V2S0Z5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,V2S0Z5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,V2S0Z5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,V2S0Z5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [255]:
surrey_grouped = surrey_onehot.groupby('Neighborhood Postal Code').mean().reset_index()
surrey_grouped

,Neighborhood Postal Code,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Big Box Store,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Stop,Business Service,Café,Carpet Store,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Food & Drink Shop,Food Court,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Library,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Multiplex,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoothie Shop,Spa,Sporting Goods Shop,Stables,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Thai Restaurant,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Wine Shop
0,V2S0Z5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.066667,0.0,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.066667,0.000000,0.000000,0.000000,0.00,0.00000,0.00000,0.066667,0.000000,0.000000,0.0,0.066667,0.000000,0.133333,0.000000,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.0,0.000000,0.000000,0.066667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.066667,0.000000,0.000000,0.133333,0.066667,0.000000,0.00
1,V2S9J2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,0.00000,0.00,0.0,0.000000,0.000000,0.142857,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,V2W5Y4,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.200000,0.00,0.00000,0.00000,0.000000,0.000000,0.000000,0.2,0.200000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.2,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,V3A8X9,0.050000,0.000000,0.000000,0.000000,0.0,0.000000,0.100000,0.000000,0.050000,0.0,0.00000,0.000000,0.000000,0.00000,0.100000,0.000000,0.0,0.050000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.100000,0.000000,0.000000,0.00,0.00000,0.0000

In [256]:
surrey_grouped.shape

(34, 97)

In [257]:
num_top_venues = 3

for hood in surrey_grouped['Neighborhood Postal Code']:
    print("----"+hood+"----")
    temp = surrey_grouped[surrey_grouped['Neighborhood Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- V2S0Z5----
                  venue  freq
0      Toy / Game Store  0.13
1  Fast Food Restaurant  0.13
2              Pharmacy  0.07


---- V2S9J2----
         venue  freq
0          Gym  0.29
1  Supermarket  0.14
2     Pharmacy  0.14


---- V2W5Y4----
               venue  freq
0  Convenience Store   0.2
1              Hotel   0.2
2        Gas Station   0.2


---- V3A8X9----
          venue  freq
0  Burger Joint   0.1
1   Coffee Shop   0.1
2          Bank   0.1


---- V3E6Y6----
              venue  freq
0       Pizza Place  0.33
1      Carpet Store  0.33
2  Storage Facility  0.33


---- V3R0L3----
         venue  freq
0  Coffee Shop  0.12
1  Pizza Place  0.06
2  Supermarket  0.06


---- V3R9M9----
               venue  freq
0  Convenience Store  0.50
1        Pizza Place  0.25
2         Restaurant  0.25


---- V3S0M6----
                 venue  freq
0          Coffee Shop  0.25
1  American Restaurant  0.05
2            Bookstore  0.05


---- V3S2J6----
                venue  freq


In [258]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [259]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

surrey_venues_sorted = pd.DataFrame(columns=columns)
surrey_venues_sorted['Neighborhood Postal Code'] = surrey_grouped['Neighborhood Postal Code']

for ind in np.arange(surrey_grouped.shape[0]):
    surrey_venues_sorted.iloc[ind, 1:] = return_most_common_venues(surrey_grouped.iloc[ind, :], num_top_venues)

surrey_venues_sorted.head()

,Neighborhood Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,V2S0Z5,Toy / Game Store,Fast Food Restaurant,Electronics Store,Diner,Pizza Place,Clothing Store,Pharmacy,Big Box Store,Japanese Restaurant,Sushi Restaurant
1,V2S9J2,Gym,Japanese Restaurant,Park,Pharmacy,Supermarket,Sandwich Place,Donut Shop,Construction & Landscaping,Convenience Store,Deli / Bodega
2,V2W5Y4,Electronics Store,Gas Station,Convenience Store,Hotel,Donut Shop,Wine Shop,Construction & Landscaping,Deli / Bodega,Department Store,Dim Sum Restaurant
3,V3A8X9,Bank,Burger Joint,Coffee Shop,Restaurant,Grocery Store,Sandwich Place,Big Box Store,Café,Electronics Store,Fast Food Restaurant
4,V3E6Y6,Pizza Place,Carpet Store,Storage Facility,Wine Shop,Donut Shop,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dim Sum Restaurant


## 5. <span style="color:orange">Creating Sunburst chart for most commonly visited top ten venues for each neighbourhood postal code in Surrey</span> 

In [260]:
import plotly.express as px
import plotly.graph_objects as go

In [261]:
num_top_venues = 10
temp_df_append = pd.DataFrame()

for hood in surrey_grouped['Neighborhood Postal Code']:
    
    temp = surrey_grouped[surrey_grouped['Neighborhood Postal Code'] == hood].T.reset_index()
    temp['Neighborhood Postal Code'] = hood
    temp.columns = ['venue','freq','Neighborhood Postal Code']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    temp_df = pd.DataFrame(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    temp_df_append = temp_df_append.append(temp_df, ignore_index=True)
   

In [262]:
temp_df_append

,venue,freq,Neighborhood Postal Code
0,Toy / Game Store,0.13,V2S0Z5
1,Fast Food Restaurant,0.13,V2S0Z5
2,Pharmacy,0.07,V2S0Z5
3,Japanese Restaurant,0.07,V2S0Z5
4,Multiplex,0.07,V2S0Z5
5,Pizza Place,0.07,V2S0Z5
6,Clothing Store,0.07,V2S0Z5
7,Electronics Store,0.07,V2S0Z5
8,Gift Shop,0.07,V2S0Z5
9,Diner,0.07,V2S0Z5


In [263]:
temp_df_append.shape

(340, 3)

In [265]:
fig=px.sunburst(temp_df_append, path=['Neighborhood Postal Code', 'venue'], values = 'freq')
fig.show()
    